In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as mpl

%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 6)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.metrics import accuracy_score

from mlfinlab.feature_importance import (
    feature_importance_mean_decrease_impurity,
    feature_importance_mean_decrease_accuracy,
    feature_importance_sfi,
    plot_feature_importance,
    get_orthogonal_features,
)

from mlfinlab.cross_validation import PurgedKFold, ml_cross_val_score
from mlfinlab.util.multiprocess import process_jobs

A few interesting notes from this chapter:

**Marcos' first law of backtesting:**

**Backtesting is not a research tool. Feature importance is.**


Once we have found what features are important, we can learn more by conducting a number of experiments.

- Are these features important all the time, or only in some specific environments?
- What triggers a change in importance over time?
- Can these regime switches be predicted?
- Are those important features also relevant to other related financial instruments?
- Ahe they relevant to other asset classes?
- What are the most relevant features across all financial instruments?
- What is the subset of features with the  highest rank correlation across the entire investment universe?



In [2]:
from sklearn.datasets import make_classification


def get_test_data(n_features=40, n_informative=10, n_redundant=10, n_samples=10000):
    # generate a random dataset for a classification problem    
    trnsX, cont = make_classification(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_redundant=n_redundant, random_state=0, shuffle=False)
    df0 = pd.DatetimeIndex(periods=n_samples, freq=pd.tseries.offsets.Minute(), end=pd.datetime.today())
    trnsX = pd.DataFrame(trnsX, index=df0)
    cont = pd.Series(cont, index=df0).to_frame('bin')
    df0 = ['I_%s' % i for i in range(n_informative)] + ['R_%s' % i for i in range(n_redundant)]
    df0 += ['N_%s' % i for i in range(n_features - len(df0))]
    trnsX.columns = df0
    cont['w'] = 1.0 / cont.shape[0]
    cont['t1'] = pd.Series(cont.index, index=cont.index)
    return trnsX, cont

In [3]:
def feature_importances(X, cont, method, allow_masking_effects=False, n_splits=10):
    max_features = None if allow_masking_effects else 1
    clf = DecisionTreeClassifier(
        criterion='entropy', max_features=max_features, class_weight='balanced', min_weight_fraction_leaf=0.0
    )
    clf = BaggingClassifier(
        base_estimator=clf, n_estimators=1000, max_features=1.0, max_samples=1.0, oob_score=True, n_jobs=-1
    )
    fit = clf.fit(X, cont['bin'])
    oob_score = fit.oob_score_

    cv_gen = PurgedKFold(n_splits=n_splits, samples_info_sets=cont['t1'])
    oos_score = ml_cross_val_score(clf, X, cont['bin'], cv_gen=cv_gen, scoring=accuracy_score).mean()

    if method == 'MDI':
        imp = feature_importance_mean_decrease_impurity(fit, X.columns)
    elif method == 'MDA':
        imp = feature_importance_mean_decrease_accuracy(clf, X, cont['bin'], cv_gen, scoring=accuracy_score)
    elif method == 'SFI':
        imp = feature_importance_sfi(clf, X, cont['bin'], cv_gen, scoring=accuracy_score)
    
    return imp, oob_score, oos_score


def test_data_func(X, cont, run='', allow_masking_effects=False, methods=['MDI', 'MDA', 'SFI']):
    for method in methods:
        feature_imp, oob_score, oos_score = feature_importances(X, cont, method, allow_masking_effects)

        plot_feature_importance(
            feature_imp, oob_score=oob_score, oos_score=oos_score,
            savefig=True, output_path='img/{}_feat_imp{}.png'.format(method, run)
        )


# 8.1a

Using the code presented in Section 8.6

Generate a dataset $(X, y)$

In [9]:
X, cont = get_test_data(n_features=12, n_informative=4, n_redundant=4, n_samples=5000)
X.head()

,I_0,I_1,I_2,I_3,R_0,R_1,R_2,R_3,N_0,N_1,N_2,N_3
2020-02-16 05:29:14.539910,-3.941539,-1.955124,-1.247683,-0.665536,2.870924,0.706670,-0.144982,-1.498281,-0.229430,0.177231,0.648948,-0.818646
2020-02-16 05:30:14.539910,-2.882175,-1.822702,-0.568862,0.103451,2.196651,0.966482,-0.527894,-1.100332,0.130209,-0.831310,1.484291,0.320911
2020-02-16 05:31:14.539910,-1.897824,-0.659752,-0.575968,1.432049,1.647345,0.800773,-0.995133,-1.899108,-1.667659,-0.005389,2.347850,0.202494
2020-02-16 05:32:14.539910,-2.574587,1.990887,0.383741,3.980372,3.637930,0.773705,-1.803899,-6.490407,0.105738,1.093880,-0.037027,-1.414238
2020-02-16 05:33:14.539910,-1.885823,-2.601728,-1.325420,-0.736274,0.621126,0.755418,-0.237697,1.156626,-1.178807,0.069023,0.454516,-0.522534


# 8.1b

Using the code presented in Section 8.6

Apply a PCA transformation on X, which we denote $\dot X$.

In [10]:
Xdot = pd.DataFrame(get_orthogonal_features(X), index=X.index).add_prefix("PCA_")
Xdot.head()

,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6
2020-02-16 05:29:14.539910,2.065379,0.146105,-1.182983,0.134590,0.530868,-0.363318,0.852387
2020-02-16 05:30:14.539910,1.839769,0.822382,-0.949730,1.217152,-0.766848,-0.026661,1.015070
2020-02-16 05:31:14.539910,1.955510,1.095325,0.280685,2.428007,0.794605,-0.732763,1.020887
2020-02-16 05:32:14.539910,4.569843,1.217288,1.651004,-1.110274,1.290850,-0.047355,0.666502
2020-02-16 05:33:14.539910,0.234743,0.652217,-1.114901,0.621042,0.784154,-0.946805,0.232960


# 8.1c

Using the code presented in Section 8.6

Compute MDI, MDA, and SFI feature importance on $(\dot X, y)$, where the base estimator is a RF.

In [ ]:
test_data_func(Xdot, cont, '_8.1c')

![title](img/MDI_feat_imp_8.1c2.png)
![title](img/MDA_feat_imp_8.1c2.png)
![title](img/SFI_feat_imp_8.1c2.png)

# 8.1d

Using the code presented in Section 8.6

Do the three methods agree on what features are important? Why?

**A: PCA successfully helped us reduce our data from 12 features to 7 and across those 7 features, our 3 feature importance methods agreed that the first few principal components (PCA_{0, 1,2}) are the most important.**

# 8.2a

From exercise 1, generate a new dataset $(\ddot X, y)$, where $\ddot X$ is a feature union of $X$ and $\dot X$.

Compute MDI, MDA, and SFI feature importance on $(\ddot X, y)$, where the base estimator is a RF.

In [11]:
Xdotdot = pd.concat([X, Xdot], axis=1)

In [ ]:
test_data_func(Xdotdot, cont, '_8.2a')

![title](img/MDI_feat_imp_8.2a.png)
![title](img/MDA_feat_imp_8.2a.png)
![title](img/SFI_feat_imp_8.2a.png)

# 8.2b

From exercise 1, generate a new dataset $(\ddot X, y)$, where $\ddot X$ is a feature union of $X$ and $\dot X$.

Do the three methods agree on what features are important? Why?

**A: MDI & SFI rank untransformed informative & redundant features above noisy ones and the first principal components over latter ones. MDA in this case does not seem to be able to rank the features correctly.**

# 8.3a

Take the results from exercise 2: 

Drop the most important features according to each method, resulting in a features matrix $\dddot X$.

In [12]:
most_important_features = ['I_2', 'PCA_1', 'PCA_0', 'R_2', 'I_1']
Xdotdotdot = Xdotdot.loc[:, ~Xdotdot.columns.isin(most_important_features)]

# 8.3b

Take the results from exercise 2: 

Compute MDI, MDA, and SFI feature importance on $(\dddot X, y)$, where the base estimator is a RF.

In [ ]:
test_data_func(Xdotdotdot, cont, '_8.3b')

# 8.3c

Take the results from exercise 2: 

Do you appreciate significant changes in the rankings of important features, relative to the results from exercise 2?

![title](img/MDI_feat_imp_8.3b.png)
![title](img/MDA_feat_imp_8.3b.png)
![title](img/SFI_feat_imp_8.3b.png)

**A: MDI & SFI seem unperturbed, while MDA has shifted a lot and now assigns positive feature importance to all remaining first principal components and informative and redundant features.**


# 8.4a

Using the code presented in Section 8.6:

Generate a dataset $(X, y)$ of 1E6 observations, where 5 features are informative, 5 are redundant and 10 are noise.

In [4]:
n_samples = 10000
X, cont = get_test_data(n_features=20, n_informative=5, n_redundant=5, n_samples=n_samples)

# 8.4b

Using the code presented in Section 8.6:

Split $(X, y)$ into 10 datasets, each of 1E5 observations.

**A: Implemented in the next answer.**

# 8.4c

Using the code presented in Section 8.6:

Compute the parallelized feature importance on each of the 10 datasets.

In [5]:
def combine_imps(imps):
    return pd.DataFrame({
        'mean': pd.concat([x['mean'] for x in imps], axis=1).mean(axis=1),
        'std': pd.concat([x['std'] for x in imps], axis=1).mean(axis=1),
    })

def chunked_test_data_func(X, cont, n_chunks=1, run='', allow_masking_effects=False, methods=['MDI', 'MDA', 'SFI']):
    from feature_importances_mp import feature_importances
    chunks = np.array_split(X.index, n_chunks)

    for method in methods:
        jobs = [{
            'func': feature_importances,
            'X': X.loc[chunk],
            'cont': cont.loc[chunk],
            'method': method,
            'allow_masking_effects': allow_masking_effects,
        } for chunk in chunks]

        results = process_jobs(jobs, num_threads=32)
    
        imps, oobs, ooss = zip(*results)

        feature_imp = combine_imps(imps)
        oob_score, oos_score = pd.Series(oobs).mean(), pd.Series(ooss).mean()

        plot_feature_importance(
            feature_imp, oob_score=oob_score, oos_score=oos_score,
            savefig=True, output_path='img/{}_feat_imp{}.png'.format(method, run)
        )



In [ ]:
chunked_test_data_func(X, cont, n_chunks=10, run='_8.4c_10chunks')

#### Parallelized feature importance:

![title](img/MDI_feat_imp_8.4c_10chunks.png)
![title](img/MDA_feat_imp_8.4c_10chunks.png)
![title](img/SFI_feat_imp_8.4c_10chunks.png)

# 8.4d

Using the code presented in Section 8.6:

Compute the stacked feature importance on the combined dataset $(X, y)$.

In [ ]:
test_data_func(X, cont, run='_8.4c_1chunk')

#### Stacked feature importance:

![title](img/MDI_feat_imp_8.4c_1chunk.png)
![title](img/MDA_feat_imp_8.4c_1chunk.png)
![title](img/SFI_feat_imp_8.4c_1chunk.png)

# 8.4e 

Using the code presented in Section 8.6:

What causes the discrepancy between the two? Which one is more reliable?

**A: Both methods generate similar rankings, with informative and redundant features above noisy ones, while the more computationally intensive (stacked) does so by a much wider margin.**

# 8.5

Repeat all MDI calculations from exercises 1-4, but this time allow for masking effects. That means, do not set `max_features=int(1)` in Snippet 8.2. How do results differ as a consequence of this change? Why?

In [ ]:
X, cont = get_test_data(n_features=12, n_informative=4, n_redundant=4, n_samples=5000)
Xdot = pd.DataFrame(get_orthogonal_features(X), index=X.index).add_prefix("PCA_")
test_data_func(Xdot, cont, '_8.5_1', allow_masking_effects=True, methods=['MDI'])

![title](img/MDI_feat_imp_8.5_1_2.png)

In [ ]:
Xdotdot = pd.concat([X, Xdot], axis=1)
test_data_func(Xdotdot, cont, '_8.5_2', allow_masking_effects=True, methods=['MDI'])

![title](img/MDI_feat_imp_8.5_2.png)

**A: There is little change for the PCA-transformed features, while MDI seems to perform a lot better on the union of transformed and non-transformed features when allowing for masking effects.**

In [ ]:
most_important_features = ['I_2', 'PCA_1', 'I_1', 'PCA_0', 'R_3']

Xdotdotdot = Xdotdot.loc[:, ~Xdotdot.columns.isin(most_important_features)]
test_data_func(Xdotdotdot, cont, '_8.5_3', allow_masking_effects=True, methods=['MDI'])

![title](img/MDI_feat_imp_8.5_3.png)

In [ ]:
n_samples = 10000
X, cont = get_test_data(n_features=20, n_informative=5, n_redundant=5, n_samples=n_samples)

chunked_test_data_func(X, cont, n_chunks=10, run='_8.5_4', allow_masking_effects=True, methods=['MDI'])

![title](img/MDI_feat_imp_8.5_4.png)

In [ ]:
test_data_func(X, cont, '_8.5_5', allow_masking_effects=True, methods=['MDI'])

![title](img/MDI_feat_imp_8.5_5.png)


**A: Allowing for masking effects still manages to rank PCA features correctly, however when untransformed redundant and noisy features are introduced, the feature importance methods quickly produce much worse results than when run when not allowing for masking effects. While stacked feature importance still does OK, parallelized feature importance also ranks some noisy above informative, and some redundant below most other features.**